# Cubic anisotropy

In [1]:
import os
import k3d
import random
import discretisedfield as df
import micromagneticmodel as mm
import micromagneticdata as md
import oommfc as oc

p1 = (-15e-9, 0, 0)
p2 = (15e-9, 1e-9, 1e-9)
n = (10, 1, 1)

region = df.Region(p1=p1, p2=p2)
mesh = df.Mesh(region, n=n)

Ms = 1e6
u1 = (1, 0, 0)
u2 = (0, 1, 0)
K = 7e5

random.seed(3)
def value_random(point):
    m = [random.random()*2 - 1 for i in range(3)]
    return m

system = mm.System(name='cubic_anisotropy')
system.energy = mm.CubicAnisotropy(K=K, u1=u1, u2=u2)
system.dynamics = mm.Damping(alpha=0.5)  # No precession to make animations faster
system.m = df.Field(mesh, dim=3, value=value_random, norm=Ms)

try:
    oc.delete(system)
except FileNotFoundError:
    pass

td = oc.TimeDriver()
td.drive(system, t=70e-12, n=250)  # The aim is to have 10s videos n=10*25=250

print(f'Average m: {system.m.orientation.average}')

system.m.k3d_vector()

Running OOMMF (ExeOOMMFRunner) [2020/06/17 20:39]... (3.6 s)
Average m: (-0.20117063840593352, 0.10051877331412676, 0.10027314596088388)


Output()

In [2]:
data = md.Data(name=system.name)

plot = k3d.plot()

@df.interact(drive_number=data.selector(),
             timestep=data.drive(0).slider(continuous_update=False))
def my_plot(drive_number, timestep):
    data.drive(drive_number).step(timestep).orientation.k3d_vector(plot=plot, interactive_field=system.m, head_size=2)

plot.display()

interactive(children=(BoundedIntText(value=0, description='drive', max=1), IntSlider(value=0, continuous_updat…

Output()

In [3]:
data.drive(0).ovf2vtk()